<img src="materials/images/introduction-to-transcriptomics-cover.png"/>

# Introduction to Transcriptomics

`🕒 This module should take less than 1 hour to complete.`

`✍️ This notebook is written using R.`

<img src="materials/images/central-dogma.png"/>


# What is the transcriptome?

The **transcriptome** is the complete set of RNA transcripts and their quantity that are produced by the genome, under specific circumstances, e.g., specific developmental stage or physiological condition, or in a specific cell.

<img src="materials/images/types_of_rna.png"/>

# What is transcriptomics?

**Transcriptomics** is the study of the transcriptome using high-throughput sequencing (HTS) methods, such as next-generation sequencing (NGS).



# What is RNA-Seq?

**RNA-Seq** (RNA Sequencing) is a revolutionary tool for transcriptomics that enables transcriptome-profiling.

**Bulk RNA-Seq** captures only an “average” of the expression profiles of thousands of cells. 

**Single-cell RNA-Seq (scRNA-Seq)** allows the capture of individual measurements across thousands of isolated single cells.

**Single-nucleus RNA-Seq (snRNA-Seq)** profiles gene expression in cells which are difficult to isolate, such as those from tissues that are archived or which are hard to be dissociated. It is an alternative to scRNA-Seq as it analyzes nuclei instead of intact cells. 

<img src="materials/images/rnaseq_overview.png"/>

# Common RNA-Seq Analysis Goals

* Transcriptome assembly and profiling
* Novel transcript discovery
* Quantify alternative splicing
* Precise measurement of levels of transcripts (genes and isoforms)
* Identification of differentially-expressed genes (DEGs) between treatments or cell populations
* Single-cell transcriptomics (scRNA-Seq) to identify novel cell types

<img src="materials/images/bulk_sc_rnaseq_contrasts.png"/>

**Bulk RNA-Seq** shows no change in expression of *Gene X* between the two samples.

**scRNA-Seq** detects a change in expression of *Gene X* between the two samples in *Cell type b* but not in *Cell type c*.

<img src="materials/images/pros_cons.png"/>

<img src="materials/images/sc_vs_sn_rnaseq.png"/>

<img src="materials/images/bulk_rnaseq_workflow.png"/>

<img src="materials/images/scrnaseq_workflow.png"/>

<img src="materials/images/file_formats.png"/>

The reference genome sequence is typically represented in FASTA format (file suffix *.fa* or *.fasta*).

The Gene Transfer Format (GTF) is a file format used to hold information about the gene structure of the reference genome (file suffix *.gtf*).

The RNA-Seq reads (sequences) from sequenced samples are typically represented in the FASTQ format (file suffix *.fq* or *.fastq*).

FASTQ files are mapped (aligned) to the reference genome to yield Sequence Alignment Map (SAM) files. Compressed binary versions of SAM files used to represent aligned sequences are referred to as BAM files (file suffix *.bam*).

<img src="materials/images/demo_background.png"/>

<img src="materials/images/demo_dataset.png"/>

<img src="materials/images/star_rsem_deseq2_pipeline.png"/>

**STAR** splice-aware mapper was used to map sample FASTQs against the human hg38 reference.

**RSEM** was used for transcript quantification to yield the gene counts file *rsem.merged.gene_counts.tsv*.

Sample metadata is in file *coldata.tsv*.

# Exercise

Let us proceed to find differentially-expressed genes (DEGs) for OHT vs. Control and DPN vs. Control contrasts using the **DESeq2** R package and generate Heatmaps and Volcano Plots for the most significant DEGs.

A **Heatmap** is a data visualization technique that shows magnitude of a phenomenon as color in two dimensions. 

A **Volcano Plot** is a type of scatterplot that shows statistical significance (P-value) versus magnitude of change (Fold Change). It is useful for identifying DEGs that differ significantly between two groups of experimental subjects.

In [ ]:
# R package imports
library("DESeq2")
library("pheatmap")
library("ggplot2")
library("EnhancedVolcano")

In [ ]:
# Set plot dimensions
options(repr.plot.width=20, repr.plot.height=10)

In [ ]:
# Read and process count matrix
countdata <- read.table('data/rsem.merged.gene_counts.tsv', header = TRUE, sep = "\t", row.names = 1) #start at column 1
countdata <- subset(countdata, select = -transcript_id.s.) #remove 'transcript_id.s.' column
countdata <- round(countdata,0) #round to integers
dim(countdata)
head(countdata)

In [ ]:
# Read and process sample metadata table
coldata <- read.table('data/coldata.tsv', header = TRUE, sep = "\t", row.names = 1) #start at column 1
coldata$condition <- factor(coldata$condition)
coldata$patient <- factor(coldata$patient)
dim(coldata)
coldata

In [ ]:
# CHECK: columns of the count matrix and the rows of the column data are in the same order
all(colnames(countdata) == rownames(coldata))

In [ ]:
# Differential Expression Analysis using DESeq2
dds <- DESeqDataSetFromMatrix(countData = countdata, colData = coldata, design = ~ patient + condition) #measure the effect of the condition controlling for patient differences
keep <- rowSums(counts(dds)) > 0 #pre-filter to remove rows that have no counts
dds <- dds[keep,]
dds <- DESeq(dds)
resultsNames(dds)
levels(dds$condition)
levels(dds$patient)
dim(dds)

In [ ]:
# Variance stabilizing transformation (VST)
vsd <- vst(dds, blind=FALSE)
head(assay(vsd))

In [ ]:
# Principal Component Analysis (PCA) of the samples
pcaData <- plotPCA(vsd, intgroup=c("condition", "patient"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=patient, shape=condition)) +
  geom_point(size=3) +
  geom_text_repel(aes(label = rownames(pcaData))) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()

In [ ]:
# Set test and control cases
test <- "OHT"
control <- "Control"

In [ ]:
# Run differential expression for selected contrast
res <- results(dds, contrast=c("condition",test,control))
resOrdered <- res[order(res$padj),] #order by adjusted pvalue (FDR)
head(resOrdered)

In [ ]:
# Define annotation columns
df <- as.data.frame(colData(dds)[,c("condition","patient")])

In [ ]:
# Heatmap of top N=20 genes by adjusted pvalue (FDR)
N <- 20
sigGenes <- head(row.names(resOrdered), N)
mat  <- assay(vsd)[sigGenes,]
selectedSamples <- rownames(subset(coldata, coldata$condition %in% c(test, control)))
mat <- mat[,colnames(mat) %in% selectedSamples]
mat <- mat - rowMeans(mat) #row mean centering
paste(test,'vs.',control,'Heatmap top',N,'genes sorted by FDR')
pheatmap(mat, cluster_rows=TRUE, show_rownames=TRUE, cluster_cols=TRUE, annotation_col=df)

In [ ]:
# Volcano Plot
EnhancedVolcano(res,
    lab = rownames(res),
    x = 'log2FoldChange',
    y = 'padj',
    xlim = c(-2, 2),
    ylim = c(0, -log10(10e-5)),
    title = paste(test,'vs.',control,'Volcano Plot'),
    pCutoff = 0.05,
    FCcutoff = 0.2,
    pointSize = 4.0,
    labSize = 6.0,
    legendPosition = "right",
    legendLabSize = 12,
    legendIconSize = 4.0,
    drawConnectors = TRUE,
    widthConnectors = 0.75,
    boxedLabels = TRUE)

In [ ]:
# Set test and control cases
test <- "DPN"
control <- "Control"

In [ ]:
# Run differential expression for selected contrast
res <- results(dds, contrast=c("condition",test,control))
resOrdered <- res[order(res$padj),] #order by adjusted pvalue (FDR)
head(resOrdered)

In [ ]:
# Heatmap of top N=20 genes by adjusted pvalue (FDR)
N <- 20
sigGenes <- head(row.names(resOrdered), N)
mat  <- assay(vsd)[sigGenes,]
selectedSamples <- rownames(subset(coldata, coldata$condition %in% c(test, control)))
mat <- mat[,colnames(mat) %in% selectedSamples]
mat <- mat - rowMeans(mat) #row mean centering
paste(test,'vs.',control,'Heatmap top',N,'genes sorted by FDR')
pheatmap(mat, cluster_rows=TRUE, show_rownames=TRUE, cluster_cols=TRUE, annotation_col=df)

In [ ]:
# Volcano Plot
EnhancedVolcano(res,
    lab = rownames(res),
    x = 'log2FoldChange',
    y = 'padj',
    xlim = c(-2, 2),
    ylim = c(0, -log10(10e-12)),
    title = paste(test,'vs.',control,'Volcano Plot'),
    pCutoff = 0.05,
    FCcutoff = 0.2,
    pointSize = 4.0,
    labSize = 6.0,
    legendPosition = "right",
    legendLabSize = 12,
    legendIconSize = 4.0,
    drawConnectors = TRUE,
    widthConnectors = 0.75,
    boxedLabels = TRUE)

# References

[1] Palazzo and Lee. doi: 10.3389/fgene.2015.00002

[2] Kukurba and Montgomery. (2015) Cold Spring Harb Protoc 2015(11):951–969

[3] Dmitry Velmeshev, UCSF

[4] Iqbal et al. Arteriosclerosis, Thrombosis, and Vascular Biology. 2021;41:585–600

[5] Method of the Year 2013. Nat Methods 11, 1 (2014)

[6] https://lizard.bio/blog/single-cell-vs-bulk/

[7] https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/03_alignment.html

[8] https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

[9] Ewels et al. Bioinformatics (2016) doi: 10.1093/bioinformatics/btw354

[10] https://www.bioinformatics.babraham.ac.uk/projects/trim_galore/

[11] Dobin et al. Bioinformatics, 2013 Jan; 29(1): 15–21

[12] Kim et al. Nature Biotechnology 37, 907–915 (2019)

[13] Patro et al. Nature Methods volume 14, 417–419 (2017)

[14] Bray et al. Nature Biotechnology 34, 525–527 (2016)

[15] Anders and Huber. Genome Biol 11, R106 (2010)

[16] Robinson and Oshlack. Genome Biol 11, R25 (2010)

[17] Liao et al. Bioinformatics, 30(7):923-30 (2014)

[18] Anders et al. bioRxiv 2014. doi: 10.1101/002824

[19] Korotkevich et al. bioRxiv 2019. doi: 10.1101/060012

[20] Li et al. (2009). 25(16):2078-9

[21] Robinson et al. Nature Biotechnology 29:24–26 (2011)

[22] https://samtools.github.io/hts-specs/SAMv1.pdf

[23] Luecken and Theis. Mol Syst Biol (2019)15:e8746

[24] Djebali et al. Methods Mol Biol 2017 1468:201-219

[25] Haglund et al. J Clin Endocrinol Metab, December 2012, 97(12):4631–4639

[26] https://www.labpedia.net/parathyroid-hormone-pth/

---

# Contributions & acknowledgment

Thank the following team to work on this module:

- **Module Content:** Ramesh V Nair
- **Engineering:** Amit Dixit
- **UX/UI Design & Illustration:** Ramesh V Nair, Kexin Cha
- **Video Production:** Francesca Goncalves
- **Project Management:** Amir Bahmani, Kexin Cha

---

Copyright (c) 2022 Stanford Data Ocean (SDO)

All rights reserved.